<a href="https://colab.research.google.com/github/Avarel/Aljava/blob/master/tutorial_lstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup
Importing **Tensorflow** and the necessities.

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  %tensorflow_version 2.x
except Exception:
  pass

!pip install tensorflow-gpu
import tensorflow as tf

import numpy as np
import os
import datetime

from tensorflow.python.client import device_lib

Checking the runtime and hardware.

In [0]:
print(device_lib.list_local_devices())
print("TensorFlow version: ", tf.__version__)

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 968094530466318055
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 10264808288884156612
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 2113702616245548143
physical_device_desc: "device: XLA_GPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 15956161332
locality {
  bus_id: 1
  links {
  }
}
incarnation: 12908716877714422554
physical_device_desc: "device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0"
]
TensorFlow version:  2.1.0-rc1


In [42]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
    print('User uploaded file {name} with size {length} bytes'.format(name = fn, length = len(uploaded[fn])))

KeyboardInterrupt: ignored

In [43]:
path = os.getcwd()

text = open(path + '/Iliad_v3.txt',  'rb').read().decode(encoding='utf-8')

print("Text is {} characters long".format(len(text)))

Text is 886809 characters long


In [44]:
words = [w for w in text.split(' ') if w.strip() != '' or w == '\n']
print("Text is {} words long".format(len(words)))

Text is 153260 words long


In [45]:
print(text[0:100])

  achilles' wrath, to greece the direful spring
  of woes unnumber'd, heavenly goddess, sing!
  that


In [46]:
vocab = sorted(set(text))
print ('There are {} unique characters'.format(len(vocab)))
char2int = {c:i for i, c in enumerate(vocab)}
int2char = np.array(vocab)
print('Vector:\n')
for char,_ in zip(char2int, range(len(vocab))):
    print(' {:4s}: {:3d},'.format(repr(char), char2int[char]))

There are 34 unique characters
Vector:

 '\n':   0,
 ' ' :   1,
 '!' :   2,
 "'" :   3,
 ',' :   4,
 '-' :   5,
 '.' :   6,
 '?' :   7,
 'a' :   8,
 'b' :   9,
 'c' :  10,
 'd' :  11,
 'e' :  12,
 'f' :  13,
 'g' :  14,
 'h' :  15,
 'i' :  16,
 'j' :  17,
 'k' :  18,
 'l' :  19,
 'm' :  20,
 'n' :  21,
 'o' :  22,
 'p' :  23,
 'q' :  24,
 'r' :  25,
 's' :  26,
 't' :  27,
 'u' :  28,
 'v' :  29,
 'w' :  30,
 'x' :  31,
 'y' :  32,
 'z' :  33,


In [48]:
text_as_int = np.array([char2int[ch] for ch in text], dtype=np.int32)
print ('{}\n mapped to integers:\n {}'.format(repr(text[:100]), text_as_int[:100]))

"  achilles' wrath, to greece the direful spring\n  of woes unnumber'd, heavenly goddess, sing!\n  that"
 mapped to integers:
 [ 1  1  8 10 15 16 19 19 12 26  3  1 30 25  8 27 15  4  1 27 22  1 14 25
 12 12 10 12  1 27 15 12  1 11 16 25 12 13 28 19  1 26 23 25 16 21 14  0
  1  1 22 13  1 30 22 12 26  1 28 21 21 28 20  9 12 25  3 11  4  1 15 12
  8 29 12 21 19 32  1 14 22 11 11 12 26 26  4  1 26 16 21 14  2  0  1  1
 27 15  8 27]


In [50]:
tr_text = text_as_int[:704000] 
val_text = text_as_int[704000:] 
print(text_as_int.shape, tr_text.shape, val_text.shape)

(886809,) (704000,) (182809,)


In [0]:
batch_size = 64
buffer_size = 10000
embedding_dim = 256
epochs = 50
seq_length = 200
examples_per_epoch = len(text)//seq_length
#lr = 0.001 #will use default for Adam optimizer
rnn_units = 1024
vocab_size = len(vocab)

In [53]:
tr_char_dataset = tf.data.Dataset.from_tensor_slices(tr_text)
val_char_dataset = tf.data.Dataset.from_tensor_slices(val_text)
tr_sequences = tr_char_dataset.batch(seq_length+1, drop_remainder=True)
val_sequences = val_char_dataset.batch(seq_length+1, drop_remainder=True)
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text
tr_dataset = tr_sequences.map(split_input_target).shuffle(buffer_size).batch(batch_size, drop_remainder=True)
val_dataset = val_sequences.map(split_input_target).shuffle(buffer_size).batch(batch_size, drop_remainder=True)
print(tr_dataset, val_dataset)

<BatchDataset shapes: ((64, 200), (64, 200)), types: (tf.int32, tf.int32)> <BatchDataset shapes: ((64, 200), (64, 200)), types: (tf.int32, tf.int32)>


In [0]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim,
            batch_input_shape=[batch_size, None]
        ),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.LSTM(rnn_units,
            return_sequences=True,
            stateful=True,
            recurrent_initializer='glorot_uniform'),
        tf.keras.layers.Dropout(0.2), 
        tf.keras.layers.LSTM(
            rnn_units,
            return_sequences=True,
            stateful=True,
            recurrent_initializer='glorot_uniform'
        ),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

model = build_model(
    vocab_size = len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=batch_size)

In [60]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (64, None, 256)           8704      
_________________________________________________________________
dropout_6 (Dropout)          (64, None, 256)           0         
_________________________________________________________________
lstm_4 (LSTM)                (64, None, 1024)          5246976   
_________________________________________________________________
dropout_7 (Dropout)          (64, None, 1024)          0         
_________________________________________________________________
lstm_5 (LSTM)                (64, None, 1024)          8392704   
_________________________________________________________________
dropout_8 (Dropout)          (64, None, 1024)          0         
_________________________________________________________________
dense_2 (Dense)              (64, None, 34)           

In [61]:
for input_example_batch, target_example_batch in tr_dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape)
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels,    logits, from_logits=True)
example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Loss:      ", example_batch_loss.numpy().mean())

(64, 200, 34)
Loss:       3.5264518


In [0]:
optimizer = tf.keras.optimizers.Adam()
model.compile(optimizer=optimizer, loss=loss)
patience = 10
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=patience)

In [66]:
checkpoint_dir = './checkpoints'+ datetime.datetime.now().strftime("_%Y.%m.%d-%H:%M:%S")
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")
checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)
history = model.fit(tr_dataset, epochs=epochs, callbacks=[checkpoint_callback, early_stop] , validation_data=val_dataset)
print ("Training stopped as there was no improvement after {} epochs".format(patience))

Train for 54 steps, validate for 14 steps
Epoch 1/50
54/54 [==============================] - 17s 313ms/step - loss: 2.9956 - val_loss: 2.7580
Epoch 2/50
54/54 [==============================] - 15s 271ms/step - loss: 2.5608 - val_loss: 2.3077
Epoch 3/50
54/54 [==============================] - 15s 272ms/step - loss: 2.1699 - val_loss: 2.0232
Epoch 4/50
54/54 [==============================] - 15s 271ms/step - loss: 1.9238 - val_loss: 1.8122
Epoch 5/50
54/54 [==============================] - 15s 273ms/step - loss: 1.7339 - val_loss: 1.6564
Epoch 6/50
54/54 [==============================] - 15s 274ms/step - loss: 1.5877 - val_loss: 1.5372
Epoch 7/50
54/54 [==============================] - 15s 271ms/step - loss: 1.4797 - val_loss: 1.4497
Epoch 8/50
54/54 [==============================] - 15s 274ms/step - loss: 1.3982 - val_loss: 1.3929
Epoch 9/50
54/54 [==============================] - 15s 273ms/step - loss: 1.3360 - val_loss: 1.3499
Epoch 10/50
54/54 [==============================

In [67]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir)) 
model.build(tf.TensorShape([1, None]))
def generate_text(model, start_string):
    
    print('Generating with seed: "' + start_string + '"')
  
    num_generate = 1000
    input_eval = [char2int[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)
    text_generated = []
    temperature = 1.0
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions,      num_samples=1)[-1,0].numpy()
        input_eval = tf.expand_dims([predicted_id], 0)
        text_generated.append(int2char[predicted_id])
    return (start_string + ''.join(text_generated))
print(generate_text(model, start_string="joy of gods"))

Generating with seed: "joy of gods"
joy of gods and woes 
  rush'd to my matrons, by my rashness rest 
  the unhappy warrior thus  hus pains 
  this lame was pured thee on the prize of men,
  and whose bright crese for ever burder 
  with all his counsels, which i juptice still'd,
  before my eyes, thy counsil both approve 
  the immortal races drink the reins he took,
  and still he lays, and lustling tith the ground.
  now grant to troy the generous warrior's fate 
  untire of mans, from man and offspring slain,
  the ten of trojans, fat from aris fell'd 
  when the great pride the crowded sceptre raged,
  his beacied troops the tears and waters stood 
  a faithful steed, that strong around his flight 
  unknown to priam's race succeeds they stand,
  wake'd o'er the deeds abidst the falling sound.
  his foe driven spears, their slaughter'd verge  with watery mountains bless'd 
  brace orous took, and trembling, swains from view,
  on the shield trojans with reproach the dame,
  or li